In [2]:
#!pip install -q transformers==4.36.2 datasets==2.20.0 scikit-learn==1.4.2 torch torchvision torchaudio

import os, json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
from tqdm import tqdm
import gc

JSON_PATH = "/content/drive/MyDrive/mava/trendyo_limited2.json"
TEXT_COL = "title"
LABEL_A = "ana_kategori"
LABEL_B = "alt_kategori_1"
LABEL_C = "alt_kategori_2"

MODEL_NAME = "dbmdz/bert-base-turkish-cased"
MAX_LEN = 64
BATCH_SIZE = 32
EPOCHS_WARMUP = 1
EPOCHS_FINETUNE = 2
LR_HEAD = 3e-4
LR_ENCODER = 2e-5
SEED = 42

torch.manual_seed(SEED)
np.random.seed(SEED)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

gc.collect()
torch.cuda.empty_cache() if torch.cuda.is_available() else None

Using device: cpu


In [2]:
df = pd.read_json(JSON_PATH)
df[TEXT_COL] = df[TEXT_COL].astype(str).str.strip()

enc_a = LabelEncoder()
enc_b = LabelEncoder()
enc_c = LabelEncoder()

y_a = enc_a.fit_transform(df[LABEL_A].astype(str))
y_b = enc_b.fit_transform(df[LABEL_B].astype(str))
y_c = enc_c.fit_transform(df[LABEL_C].astype(str))

num_a = len(enc_a.classes_)
num_b = len(enc_b.classes_)
num_c = len(enc_c.classes_)

print(f"Sınıf sayıları → ana:{num_a}, alt1:{num_b}, alt2:{num_c}")

os.makedirs("/content/drive/MyDrive/mava/artifacts", exist_ok=True)

gc.collect()

Sınıf sayıları → ana:9, alt1:57, alt2:440


15

In [3]:
def create_hierarchical_categories(df, enc_a, enc_b, enc_c):
    hierarchical_structure = {}

    for idx, row in df.iterrows():
        ana_cat = row[LABEL_A]
        alt1_cat = row[LABEL_B]
        alt2_cat = row[LABEL_C]

        if ana_cat not in hierarchical_structure:
            hierarchical_structure[ana_cat] = {}

        if alt1_cat not in hierarchical_structure[ana_cat]:
            hierarchical_structure[ana_cat][alt1_cat] = []

        if alt2_cat not in hierarchical_structure[ana_cat][alt1_cat]:
            hierarchical_structure[ana_cat][alt1_cat].append(alt2_cat)

    return hierarchical_structure

hierarchical_cats = create_hierarchical_categories(df, enc_a, enc_b, enc_c)

with open("/content/drive/MyDrive/mava/artifacts/label_maps.json", "w", encoding="utf-8") as f:
    json.dump({
        "hierarchical_categories": hierarchical_cats,
        "flat_categories": {
            "ana_kategori": enc_a.classes_.tolist(),
            "alt_kategori_1": enc_b.classes_.tolist(),
            "alt_kategori_2": enc_c.classes_.tolist(),
        }
    }, f, ensure_ascii=False, indent=2)

print("Label maps kaydedildi!")

del hierarchical_cats
gc.collect()

Label maps kaydedildi!


33

In [4]:
train_idx, val_idx = train_test_split(
    np.arange(len(df)),
    test_size=0.1,
    random_state=SEED,
    stratify=y_a
)

df_train = df.iloc[train_idx]
df_val = df.iloc[val_idx]
y_a_tr, y_a_val = y_a[train_idx], y_a[val_idx]
y_b_tr, y_b_val = y_b[train_idx], y_b[val_idx]
y_c_tr, y_c_val = y_c[train_idx], y_c[val_idx]

print(f"Train size: {len(df_train)}, Val size: {len(df_val)}")

del df
gc.collect()

Train size: 642878, Val size: 71431


0

In [5]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

class ProductDataset(Dataset):
    def __init__(self, texts, labels_a, labels_b, labels_c, tokenizer, max_len):
        self.texts = texts
        self.labels_a = labels_a
        self.labels_b = labels_b
        self.labels_c = labels_c
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts.iloc[idx])

        encoding = self.tokenizer.encode_plus(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label_a': torch.tensor(self.labels_a[idx], dtype=torch.long),
            'label_b': torch.tensor(self.labels_b[idx], dtype=torch.long),
            'label_c': torch.tensor(self.labels_c[idx], dtype=torch.long)
        }

print("Dataset sınıfı tanımlandı!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Dataset sınıfı tanımlandı!


In [6]:
train_dataset = ProductDataset(
    df_train[TEXT_COL], y_a_tr, y_b_tr, y_c_tr, tokenizer, MAX_LEN
)
val_dataset = ProductDataset(
    df_val[TEXT_COL], y_a_val, y_b_val, y_c_val, tokenizer, MAX_LEN
)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

def calc_class_weights(y, num_classes):
    cw = compute_class_weight(
        class_weight="balanced",
        classes=np.arange(num_classes),
        y=y
    )
    return torch.tensor(cw, dtype=torch.float32).to(device)

class_weights_a = calc_class_weights(y_a_tr, num_a)
class_weights_b = calc_class_weights(y_b_tr, num_b)
class_weights_c = calc_class_weights(y_c_tr, num_c)

print(f"DataLoaders hazır - Train batches: {len(train_loader)}, Val batches: {len(val_loader)}")

del df_train, df_val
gc.collect()

DataLoaders hazır - Train batches: 20090, Val batches: 2233


129

In [7]:
class MultiHeadBertClassifier(nn.Module):
    def __init__(self, model_name, num_classes_a, num_classes_b, num_classes_c, dropout=0.2):
        super(MultiHeadBertClassifier, self).__init__()

        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(dropout)

        hidden_size = self.bert.config.hidden_size
        self.proj = nn.Linear(hidden_size, hidden_size)
        self.tanh = nn.Tanh()

        self.head_a = nn.Linear(hidden_size, num_classes_a)
        self.head_b = nn.Linear(hidden_size, num_classes_b)
        self.head_c = nn.Linear(hidden_size, num_classes_c)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)

        pooled_output = outputs.pooler_output
        if pooled_output is None:
            pooled_output = outputs.last_hidden_state[:, 0]

        x = self.proj(pooled_output)
        x = self.tanh(x)
        x = self.dropout(x)

        logits_a = self.head_a(x)
        logits_b = self.head_b(x)
        logits_c = self.head_c(x)

        return logits_a, logits_b, logits_c

print("Model sınıfı tanımlandı!")

Model sınıfı tanımlandı!


In [8]:
model = MultiHeadBertClassifier(MODEL_NAME, num_a, num_b, num_c)
model.to(device)

criterion_a = nn.CrossEntropyLoss(weight=class_weights_a)
criterion_b = nn.CrossEntropyLoss(weight=class_weights_b)
criterion_c = nn.CrossEntropyLoss(weight=class_weights_c)

print("Model GPU'ya yüklendi!")
print(f"Model parametreleri: {sum(p.numel() for p in model.parameters()):,}")

gc.collect()
torch.cuda.empty_cache() if torch.cuda.is_available() else None

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want t

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Model GPU'ya yüklendi!
Model parametreleri: 111,597,050


In [9]:
def train_epoch(model, data_loader, optimizer, scheduler, device, freeze_bert=False):
    model.train()
    total_loss = 0
    correct_a = correct_b = correct_c = 0
    total = 0

    for batch in tqdm(data_loader):
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels_a = batch['label_a'].to(device)
        labels_b = batch['label_b'].to(device)
        labels_c = batch['label_c'].to(device)

        logits_a, logits_b, logits_c = model(input_ids, attention_mask)

        loss_a = criterion_a(logits_a, labels_a)
        loss_b = criterion_b(logits_b, labels_b)
        loss_c = criterion_c(logits_c, labels_c)
        loss = loss_a + loss_b + loss_c

        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

        _, pred_a = torch.max(logits_a.data, 1)
        _, pred_b = torch.max(logits_b.data, 1)
        _, pred_c = torch.max(logits_c.data, 1)

        correct_a += (pred_a == labels_a).sum().item()
        correct_b += (pred_b == labels_b).sum().item()
        correct_c += (pred_c == labels_c).sum().item()
        total += labels_a.size(0)

    return (total_loss / len(data_loader),
            correct_a / total,
            correct_b / total,
            correct_c / total)

def eval_model(model, data_loader, device):
    model.eval()
    total_loss = 0
    correct_a = correct_b = correct_c = 0
    total = 0

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels_a = batch['label_a'].to(device)
            labels_b = batch['label_b'].to(device)
            labels_c = batch['label_c'].to(device)

            logits_a, logits_b, logits_c = model(input_ids, attention_mask)

            loss_a = criterion_a(logits_a, labels_a)
            loss_b = criterion_b(logits_b, labels_b)
            loss_c = criterion_c(logits_c, labels_c)
            loss = loss_a + loss_b + loss_c

            total_loss += loss.item()

            _, pred_a = torch.max(logits_a.data, 1)
            _, pred_b = torch.max(logits_b.data, 1)
            _, pred_c = torch.max(logits_c.data, 1)

            correct_a += (pred_a == labels_a).sum().item()
            correct_b += (pred_b == labels_b).sum().item()
            correct_c += (pred_c == labels_c).sum().item()
            total += labels_a.size(0)

    return (total_loss / len(data_loader),
            correct_a / total,
            correct_b / total,
            correct_c / total)

print("Training fonksiyonları tanımlandı!")

Training fonksiyonları tanımlandı!


In [10]:
if EPOCHS_WARMUP > 0:
    print("Phase 1: Warmup (BERT frozen) başlıyor...")

    for param in model.bert.parameters():
        param.requires_grad = False

    head_params = list(model.proj.parameters()) + list(model.head_a.parameters()) + \
                  list(model.head_b.parameters()) + list(model.head_c.parameters())

    optimizer_warmup = optim.Adam(head_params, lr=LR_HEAD)
    scheduler_warmup = get_linear_schedule_with_warmup(
        optimizer_warmup,
        num_warmup_steps=0,
        num_training_steps=len(train_loader) * EPOCHS_WARMUP
    )

    best_acc = 0
    for epoch in range(EPOCHS_WARMUP):
        print(f"\nWarmup Epoch {epoch+1}/{EPOCHS_WARMUP}")

        train_loss, train_acc_a, train_acc_b, train_acc_c = train_epoch(
            model, train_loader, optimizer_warmup, scheduler_warmup, device
        )
        val_loss, val_acc_a, val_acc_b, val_acc_c = eval_model(model, val_loader, device)

        print(f'Train Loss: {train_loss:.4f}, Train Acc A: {train_acc_a:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Acc A: {val_acc_a:.4f}')

        if val_acc_a > best_acc:
            best_acc = val_acc_a
            torch.save(model.state_dict(), '/content/drive/MyDrive/mava/artifacts/model_warmup.pth')
            print("Warmup model kaydedildi!")

    del optimizer_warmup, scheduler_warmup
    gc.collect()
    torch.cuda.empty_cache() if torch.cuda.is_available() else None

    print("Warmup phase tamamlandı!")

Phase 1: Warmup (BERT frozen) başlıyor...

Warmup Epoch 1/1


100%|██████████| 20090/20090 [39:36<00:00,  8.45it/s]


Train Loss: 10.5967, Train Acc A: 0.3436
Val Loss: 8.9240, Val Acc A: 0.5122
Warmup model kaydedildi!
Warmup phase tamamlandı!


In [ ]:
if EPOCHS_FINETUNE > 0:
    print("Phase 2: Fine-tuning (BERT unfrozen) başlıyor...")

    for param in model.bert.parameters():
        param.requires_grad = True

    optimizer_finetune = optim.Adam([
        {'params': model.bert.parameters(), 'lr': LR_ENCODER},
        {'params': model.proj.parameters(), 'lr': LR_HEAD},
        {'params': model.head_a.parameters(), 'lr': LR_HEAD},
        {'params': model.head_b.parameters(), 'lr': LR_HEAD},
        {'params': model.head_c.parameters(), 'lr': LR_HEAD}
    ])

    scheduler_finetune = get_linear_schedule_with_warmup(
        optimizer_finetune,
        num_warmup_steps=0,
        num_training_steps=len(train_loader) * EPOCHS_FINETUNE
    )

    best_acc = 0
    for epoch in range(EPOCHS_FINETUNE):
        print(f"\nFine-tune Epoch {epoch+1}/{EPOCHS_FINETUNE}")

        train_loss, train_acc_a, train_acc_b, train_acc_c = train_epoch(
            model, train_loader, optimizer_finetune, scheduler_finetune, device
        )
        val_loss, val_acc_a, val_acc_b, val_acc_c = eval_model(model, val_loader, device)

        print(f'Train Loss: {train_loss:.4f}, Train Acc A: {train_acc_a:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Acc A: {val_acc_a:.4f}')

        if val_acc_a > best_acc:
            best_acc = val_acc_a
            torch.save(model.state_dict(), '/content/drive/MyDrive/mava/artifacts/model_finetune.pth')
            print("Fine-tune model kaydedildi!")

    del optimizer_finetune, scheduler_finetune
    gc.collect()
    torch.cuda.empty_cache() if torch.cuda.is_available() else None

    print("Fine-tuning phase tamamlandı!")

Phase 2: Fine-tuning (BERT unfrozen) başlıyor...

Fine-tune Epoch 1/2


100%|██████████| 20090/20090 [1:51:36<00:00,  3.00it/s]


Train Loss: 2.7588, Train Acc A: 0.8682
Val Loss: 1.8453, Val Acc A: 0.9164
Fine-tune model kaydedildi!

Fine-tune Epoch 2/2


 41%|████      | 8182/20090 [45:26<1:05:32,  3.03it/s]

In [4]:
torch.save(model.state_dict(), '/content/drive/MyDrive/mava/artifacts/berturk_multihead_pytorch.pth')
print("Final model kaydedildi!")

id2a = {i: c for i, c in enumerate(enc_a.classes_)}
id2b = {i: c for i, c in enumerate(enc_b.classes_)}
id2c = {i: c for i, c in enumerate(enc_c.classes_)}

def predict_titles(titles, topk=1):
    model.eval()
    results = []

    with torch.no_grad():
        for title in titles:
            encoding = tokenizer.encode_plus(
                title,
                truncation=True,
                padding='max_length',
                max_length=MAX_LEN,
                return_tensors='pt'
            )

            input_ids = encoding['input_ids'].to(device)
            attention_mask = encoding['attention_mask'].to(device)

            logits_a, logits_b, logits_c = model(input_ids, attention_mask)

            probs_a = torch.softmax(logits_a, dim=1).cpu().numpy()[0]
            probs_b = torch.softmax(logits_b, dim=1).cpu().numpy()[0]
            probs_c = torch.softmax(logits_c, dim=1).cpu().numpy()[0]

            top_a = np.argsort(probs_a)[::-1][:topk]
            top_b = np.argsort(probs_b)[::-1][:topk]
            top_c = np.argsort(probs_c)[::-1][:topk]

            results.append({
                "title": title,
                "ana_top": [(id2a[j], float(probs_a[j])) for j in top_a],
                "alt1_top": [(id2b[j], float(probs_b[j])) for j in top_b],
                "alt2_top": [(id2c[j], float(probs_c[j])) for j in top_c],
            })

    return results

print("Prediction fonksiyonu hazır!")

Prediction fonksiyonu hazır!


In [23]:
# COMPLETE RESUME SCRIPT - Yeni session için
import os, json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
from tqdm import tqdm
import gc

CSV_PATH = "/content/drive/MyDrive/mava/trendyo_limited2.json"
TEXT_COL = "title"
LABEL_A = "ana_kategori"
LABEL_B = "alt_kategori_1"
LABEL_C = "alt_kategori_2"

MODEL_NAME = "dbmdz/bert-base-turkish-cased"
MAX_LEN = 64
BATCH_SIZE = 32
EPOCHS_WARMUP = 1
EPOCHS_FINETUNE = 2
LR_HEAD = 3e-4
LR_ENCODER = 2e-5
SEED = 42

torch.manual_seed(SEED)
np.random.seed(SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

print("Label maps yükleniyor...")
with open("/content/drive/MyDrive/mava/artifacts/label_maps.json", "r", encoding="utf-8") as f:
    label_maps = json.load(f)

enc_a = LabelEncoder()
enc_b = LabelEncoder()
enc_c = LabelEncoder()
enc_a.classes_ = np.array(label_maps["flat_categories"]["ana_kategori"])
enc_b.classes_ = np.array(label_maps["flat_categories"]["alt_kategori_1"])
enc_c.classes_ = np.array(label_maps["flat_categories"]["alt_kategori_2"])

num_a, num_b, num_c = len(enc_a.classes_), len(enc_b.classes_), len(enc_c.classes_)
print(f"Sınıf sayıları → ana:{num_a}, alt1:{num_b}, alt2:{num_c}")

df = pd.read_json(CSV_PATH)
df[TEXT_COL] = df[TEXT_COL].astype(str).str.strip()

y_a = enc_a.transform(df[LABEL_A].astype(str))
y_b = enc_b.transform(df[LABEL_B].astype(str))
y_c = enc_c.transform(df[LABEL_C].astype(str))

train_idx, val_idx = train_test_split(
    np.arange(len(df)), test_size=0.1, random_state=SEED, stratify=y_a
)

df_train = df.iloc[train_idx]
df_val = df.iloc[val_idx]
y_a_tr, y_a_val = y_a[train_idx], y_a[val_idx]
y_b_tr, y_b_val = y_b[train_idx], y_b[val_idx]
y_c_tr, y_c_val = y_c[train_idx], y_c[val_idx]

del df
gc.collect()
print(f"Train: {len(df_train)}, Val: {len(df_val)}")

print("Tokenizer yükleniyor...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

class ProductDataset(Dataset):
    def __init__(self, texts, labels_a, labels_b, labels_c, tokenizer, max_len):
        self.texts = texts
        self.labels_a = labels_a
        self.labels_b = labels_b
        self.labels_c = labels_c
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts.iloc[idx])
        encoding = self.tokenizer.encode_plus(
            text, truncation=True, padding='max_length',
            max_length=self.max_len, return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label_a': torch.tensor(self.labels_a[idx], dtype=torch.long),
            'label_b': torch.tensor(self.labels_b[idx], dtype=torch.long),
            'label_c': torch.tensor(self.labels_c[idx], dtype=torch.long)
        }

train_dataset = ProductDataset(df_train[TEXT_COL], y_a_tr, y_b_tr, y_c_tr, tokenizer, MAX_LEN)
val_dataset = ProductDataset(df_val[TEXT_COL], y_a_val, y_b_val, y_c_val, tokenizer, MAX_LEN)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

del df_train, df_val
gc.collect()

def calc_class_weights(y, num_classes):
    cw = compute_class_weight("balanced", classes=np.arange(num_classes), y=y)
    return torch.tensor(cw, dtype=torch.float32).to(device)

class_weights_a = calc_class_weights(y_a_tr, num_a)
class_weights_b = calc_class_weights(y_b_tr, num_b)
class_weights_c = calc_class_weights(y_c_tr, num_c)

class MultiHeadBertClassifier(nn.Module):
    def __init__(self, model_name, num_classes_a, num_classes_b, num_classes_c, dropout=0.2):
        super(MultiHeadBertClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(dropout)
        hidden_size = self.bert.config.hidden_size
        self.proj = nn.Linear(hidden_size, hidden_size)
        self.tanh = nn.Tanh()
        self.head_a = nn.Linear(hidden_size, num_classes_a)
        self.head_b = nn.Linear(hidden_size, num_classes_b)
        self.head_c = nn.Linear(hidden_size, num_classes_c)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        if pooled_output is None:
            pooled_output = outputs.last_hidden_state[:, 0]
        x = self.proj(pooled_output)
        x = self.tanh(x)
        x = self.dropout(x)
        return self.head_a(x), self.head_b(x), self.head_c(x)

print("Model yükleniyor...")
model = MultiHeadBertClassifier(MODEL_NAME, num_a, num_b, num_c)
model.to(device)

criterion_a = nn.CrossEntropyLoss(weight=class_weights_a)
criterion_b = nn.CrossEntropyLoss(weight=class_weights_b)
criterion_c = nn.CrossEntropyLoss(weight=class_weights_c)

id2a = {i: c for i, c in enumerate(enc_a.classes_)}
id2b = {i: c for i, c in enumerate(enc_b.classes_)}
id2c = {i: c for i, c in enumerate(enc_c.classes_)}

print(f"Model GPU'da: {next(model.parameters()).device}")
gc.collect()
torch.cuda.empty_cache() if torch.cuda.is_available() else None

Device: cpu
Label maps yükleniyor...
Sınıf sayıları → ana:9, alt1:57, alt2:440
Train: 642878, Val: 71431
Tokenizer yükleniyor...
Model yükleniyor...
Model GPU'da: cpu


In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

if torch.cuda.is_available():
    model.load_state_dict(torch.load('/content/drive/MyDrive/mava/artifacts/model_finetune.pth'))
else:
    model.load_state_dict(torch.load('/content/drive/MyDrive/mava/artifacts/model_finetune.pth',
                                    map_location=torch.device('cpu')))

model.to(device)
print(f"Model yüklendi ve {device}'a taşındı!")

def predict_titles(titles, topk=1):
    model.eval()
    results = []
    with torch.no_grad():
        for title in titles:
            encoding = tokenizer.encode_plus(
                title, truncation=True, padding='max_length',
                max_length=MAX_LEN, return_tensors='pt'
            )
            input_ids = encoding['input_ids'].to(device)
            attention_mask = encoding['attention_mask'].to(device)

            logits_a, logits_b, logits_c = model(input_ids, attention_mask)

            probs_a = torch.softmax(logits_a, dim=1).cpu().numpy()[0]
            probs_b = torch.softmax(logits_b, dim=1).cpu().numpy()[0]
            probs_c = torch.softmax(logits_c, dim=1).cpu().numpy()[0]

            top_a = np.argsort(probs_a)[::-1][:topk]
            top_b = np.argsort(probs_b)[::-1][:topk]
            top_c = np.argsort(probs_c)[::-1][:topk]

            results.append({
                "title": title,
                "ana_top": [(id2a[j], float(probs_a[j])) for j in top_a],
                "alt1_top": [(id2b[j], float(probs_b[j])) for j in top_b],
                "alt2_top": [(id2c[j], float(probs_c[j])) for j in top_c],
            })
    return results

sample_titles = [
    "bebek bezi ",
    "mitoloji kitap",
    " kadın gümüş kolye",
    "erkek gümüş kolye",
    "bulaşık makinesi yedek parça",
    "renkli tablo",
    "sandalye mutfak 4lü"
]

print("Test başlıyor...")
preds = predict_titles(sample_titles, topk=3)

for p in preds:
    print(f"\n  {p['title']}")
    print(f"ANA: {p['ana_top'][0][0]} ({p['ana_top'][0][1]:.3f})")
    print(f"ALT1: {p['alt1_top'][0][0]} ({p['alt1_top'][0][1]:.3f})")
    print(f"ALT2: {p['alt2_top'][0][0]} ({p['alt2_top'][0][1]:.3f})")


Device: cpu
Model yüklendi ve cpu'a taşındı!
Test başlıyor...

  bebek bezi 
ANA: Anne & Çocuk (0.984)
ALT1: Bebek Bakım (0.910)
ALT2: Bebek Bakım ve Kozmetik (0.632)

  mitoloji kitap
ANA: Kitap & Kırtasiye & Hobi (0.998)
ALT1: Din Ve Mitoloji (0.836)
ALT2: Mitoloji (0.968)

   kadın gümüş kolye
ANA: Kadın (0.687)
ALT1: Aksesuar (0.961)
ALT2: Takı & Mücevher (0.459)

  erkek gümüş kolye
ANA: Erkek (0.996)
ALT1: Aksesuar (0.991)
ALT2: Takı & Mücevher (0.971)

  bulaşık makinesi yedek parça
ANA: Elektronik (0.576)
ALT1: Beyaz Eşya (0.661)
ALT2: Bulaşık Yıkama (0.360)

  renkli tablo
ANA: Ev & Mobilya (0.974)
ALT1: Ev Dekorasyon (0.935)
ALT2: Duvar Dekorasyon Ürünü (0.426)

  sandalye mutfak 4lü
ANA: Ev & Mobilya (0.977)
ALT1: Mobilya (0.996)
ALT2: Yemek Odası (0.571)
